In [1]:
!pip install gspread
!pip install google-auth
!pip install gspread-dataframe

In [2]:
import pandas as pd # スプレッドシートから得たデータをデータフレームに変換する機能をインポートし、省略してpdと呼べるようにas pdを付ける
import gspread # スプレッドシートのデータを扱うライブラリをインポート
from google.oauth2.service_account import Credentials # スプレッドシートの認証機能をインポート
from gspread_dataframe import set_with_dataframe # スプレッドシートのデータとpandasライブラリのデータを紐づける機能をインポート

from dotenv import load_dotenv #envファイルを用いた情報を隠す方法を連携
load_dotenv()
import os

In [3]:
# 認証のために機能役割を決めるアクセス先をscopesに設定
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]
#.envで設定した変数を取得する
sp_sheet_key = os.getenv('sp_sheet_key')
service_account_file = os.getenv('service_account_file')

# その役割の許可をもらうAPIキーをservice_account_new.jsonから読み込み、credentialsに代入
# 認証キーを使うアクセス先をscopesに代入
credentials = Credentials.from_service_account_file(
    service_account_file, #Google CloudからDLした認証キーのファイルを設定する
    scopes=scopes
)

# 認証情報を格納しているcredentialsを使って、gspread.authorizeでスプレッドシートの使用許可を取り、その認証結果をgcに代入
gc = gspread.authorize(credentials)

In [4]:
# 使用するスプレッドシートのアクセス先をSP_SHEET_KEYに代入
# 開きたいスプレッドシートを認証結果を格納したgcを使ってgc.open_by_keyで開く
sh = gc.open_by_key(sp_sheet_key)

# 参照するシート名をSP_SHEETに代入
SP_SHEET = 'suumo_data'

# gc.open_by_keyで開いたスプレッドシートのsampleシートをsh.worksheet(SP_SHEET)で情報を得て、worksheetに代入する
worksheet = sh.worksheet(SP_SHEET)

In [5]:
#csvデータの読み込み
df = pd.read_csv('suumo_data_cleansing.csv') #前回クレンジングを行ったcsvデータを読み込む
df

df_upload = df # スクレイピングで取得した新しいデータdfをdf_newに代入

set_with_dataframe(sh.worksheet(SP_SHEET), df_upload, include_index=False) # set_with_dataframe(スプレッドシート名,対象データ,データフレームで自動生成されるindex数字を含むかどうか)